## 0. Description

#### DESCRIPTION: 		
##### This file scrapes Fair Price SHop (FPS) transactions for all districts of Bihar for the month of November 2024.

In [ ]:
# Importing all the required selenium libraries 
import pandas as pd
from bs4 import BeautifulSoup
import requests, lxml, os, json
import unittest
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException,StaleElementReferenceException
import re
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import Select
import time

In [ ]:
# Importing all the required selenium libraries 
import pandas as pd
from bs4 import BeautifulSoup
import requests, lxml, os, json
import unittest
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException,StaleElementReferenceException
from selenium.common.exceptions import WebDriverException
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
import pytesseract
from PIL import Image
import re
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.support.select import Select
from selenium.common.exceptions import UnexpectedAlertPresentException
import time
from python3_anticaptcha import ImageToTextTask
# import cv2 as cv2
import numpy as np
import matplotlib as plt
import csv
import html5lib

In [ ]:
import io
from io import StringIO

In [ ]:
import shutil

In [ ]:
from pathlib import Path

In [ ]:
chrome_options = Options()
#argument to switch off suid sandBox and no sandBox in Chrome 
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-setuid-sandbox")
chrome_options.add_experimental_option("prefs", {
    "download.prompt_for_download": False,  # Disable download prompt
    "download.directory_upgrade": True,     # Ensure multiple downloads go to the same directory
    "safebrowsing.enabled": True,           # Disable Safe Browsing prompt
    "profile.default_content_settings.popups": 0,
    "profile.default_content_setting_values.automatic_downloads": 1  # Allow multiple downloads
})
driver = webdriver.Chrome(options=chrome_options)

In [ ]:
#Set up file paths
db_input_folder = Path('/Users/swathi/Library/CloudStorage/Dropbox/Bihar Gates Team/PDS/admin_data/Data/FPS_Transactions/October_2024')
fps_summary_folder = Path('FPS_Summary')
transactions_folder = Path('FPS_Transactions')
downloads_folder = Path('/Users/swathi/Downloads')

In [ ]:
#Open the website
url = 'https://epos.bihar.gov.in/AbstractTransReport.jsp'
driver.get(url)
wait = WebDriverWait(driver, 20)

In [ ]:
#Go to date in September
#Using JavaScript querySelector, set the date
date = '25-10-2024'
date_download = '25.10.2024'
driver.execute_script("document.querySelector('#trans_date').value = '25-10-2024';")

#Click submit button (using CSS selector)
submit_button_selector = '#container > div:nth-child(1) > table > tbody > tr:nth-child(3) > td > button:nth-child(1)'
submit_button = driver.find_element(By.CSS_SELECTOR, submit_button_selector)
submit_button.click()

In [ ]:
# District: Araria, Index: 0
# District: Arwal, Index: 1
# District: Aurangabad, Index: 2
# District: Banka, Index: 3
# District: Begusarai, Index: 4
# District: Bhagalpur, Index: 5
# District: Bhojpur, Index: 6
# District: Buxar, Index: 7
# District: Darbhanga, Index: 8
# District: Gaya, Index: 9
# District: Gopalganj, Index: 10
# District: Jamui, Index: 11
# District: Jehanabad, Index: 12
# District: Kaimur (Bhabua), Index: 13
# District: Katihar, Index: 14
# District: Khagaria, Index: 15
# District: Kishanganj, Index: 16
# District: Lakhisarai, Index: 17
# District: Madhepura, Index: 18
# District: Madhubani, Index: 19
# District: Munger, Index: 20
# District: Muzaffarpur, Index: 21
# District: Nalanda, Index: 22
# District: Nawada, Index: 23
# District: Pashchim Champaran, Index: 24
# District: Patna, Index: 25
# District: Purbi Champaran, Index: 26
# District: Purnia, Index: 27
# District: Rohtas, Index: 28
# District: Saharsa, Index: 29
# District: Samastipur, Index: 30
# District: Saran, Index: 31
# District: Sheikhpura, Index: 32
# District: Sheohar, Index: 33
# District: Sitamarhi, Index: 34
# District: Siwan, Index: 35
# District: Supaul, Index: 36
# District: Vaishali, Index: 37

In [ ]:
#Find district elements
try:
    district_elements = wait.until(EC.visibility_of_all_elements_located((By.XPATH, "//a[contains(@onclick, 'detailsERR')]")))
    #Find district name looping through elements
    # for dist_el in district_elements:
    for dist_index in range(21, len(district_elements)):
        if dist_index==22:
            raise ValueError('District 21 is done')
        district_elements = wait.until(EC.visibility_of_all_elements_located((By.XPATH, "//a[contains(@onclick, 'detailsERR')]")))
        dist_execute_retries = 0
        district_success = False
        while dist_execute_retries<3:
            try:
                # district_name = dist_el.text.strip()
                district_name = district_elements[dist_index].text.strip()
                print(f"District: {district_name}")
            
                #Go to page of each
                # driver.execute_script(dist_el.get_attribute('onclick'))
                driver.execute_script(district_elements[dist_index].get_attribute('onclick'))
                time.sleep(0.5)
                district_success = True
                break #If successful, no more retries
    
            except (StaleElementReferenceException, NoSuchElementException) as e:
                # print(f"Could not go to block page for district {dist_el}: {e}")
                print(f"Could not go to block page for district index {dist_index}: {e}")
                break #Break loop and go to next district
    
            except (WebDriverException, TimeoutException) as e:
                # print(f"Error connecting to internet to go to block page for district {dist_el}, retrying: {e}")
                print(f"Error connecting to internet to go to block page for district index {dist_index}, retrying: {e}")
                dist_execute_retries+=1
                time.sleep(10)

        if not district_success:
            if dist_execute_retries==3:
                # print(f"Error connecting to internet to go to block page for district {dist_el}: {e}. STOPPED")
                print(f"Error connecting to internet to go to block page for district {dist_index}: {e}. STOPPED")
                # driver.close() #Close the driver
            continue #Go to the next district - don't go to block-level drilldown
    
        #Find block elements
        try:
            block_elements = wait.until(EC.visibility_of_all_elements_located((By.XPATH, "//a[contains(@onclick, 'detailsERRR')]")))
            len_block_elements = len(block_elements)
            #Loop through block elements
            # for bl_el in block_elements:
            for bl_index in range(10, len_block_elements):
                # if bl_index==9:
                #     raise ValueError('Block 8 done')
                block_elements = wait.until(EC.visibility_of_all_elements_located((By.XPATH, "//a[contains(@onclick, 'detailsERRR')]")))
                bl_execute_retries = 0
                block_success = False
                while bl_execute_retries<3:
                    try:
                        # block_name = bl_el.text.strip()
                        block_name = block_elements[bl_index].text.strip()
                        print(f"Block: {block_name}")
                
                        #Go to page of each block
                        # driver.execute_script(bl_el.get_attribute('onclick'))
                        driver.execute_script(block_elements[bl_index].get_attribute('onclick'))
                        block_success = True
                        break #No more retries if successful
                        # time.sleep(0.5)
                        
                    except (StaleElementReferenceException, NoSuchElementException) as e:
                        # print(f"Could not go to FPS page for block {block_name} in district {dist_el}: {e}")
                        print(f"Could not go to FPS page for block index {bl_index} in district index {dist_index}: {e}")
                        break #Break out of retries

                    except (WebDriverException, TimeoutException) as e:
                        # print(f"Error connecting to internet to FPS page for block {block_name} in district {dist_el}, retrying: {e}")
                        print(f"Error connecting to internet to FPS page for block index {bl_index} in district index {dist_index}, retrying: {e}")
                        # break #Go back to the block elements page. If that can't be found, it will keep retrying. not a good idea because it will then start again from first block
                        bl_execute_retries+=1
                        time.sleep(10)

                    except IndexError as e:
                        print(f"Block index {bl_index} out of index for district index {dist_index} with block elements {len_block_elements}, retrying")
                        break #Break out of retries
                
                if not block_success:
                    if bl_execute_retries==3:
                        # print(f"Error connecting to internet to FPS page for block {block_name} in district {dist_el}. STOPPED")
                        print(f"Error connecting to internet to FPS page for block index {bl_index} in district index {dist_index}. STOPPED")
                        # driver.close() #Close the driver
                    continue #Go to next block - don't drilldown further

                #Capture the FPS summary page for the block and export it to csv
                block_fps_data = []
                try:
                    block_fps_elements_clickable = wait.until(EC.visibility_of_all_elements_located((By.XPATH, "//a[contains(@onclick, 'detailsRFL')]")))
                    len_block_fps_elements_clickable = len(block_fps_elements_clickable)

                    #Scrape the data
                    # block_trans_header = driver.find_elements(By.XPATH, '//*[@id="mlstab4"]/thead/tr')
                    # #Capture header rows
                    # for row in block_trans_header:
                    #     header = [cell.text.strip() for cell in row.find_elements(By.TAG_NAME, 'th')]
                    #     # print(header)
                    #     print(len(header))
                    # block_trans_elements = driver.find_elements(By.XPATH, '//*[@id="mlstab4"]/tbody/tr')

                    #Download to Excel
                    download_retries = 0
                    while download_retries<3:
                        try:
                            driver.execute_script("tableToExcel('mlstab4', 'Detailed Trans FPS')")
                            time.sleep(1)
                            
                            #Move file from downloads into the FPS_Summary folder
                            try:
                                download_file = Path('download.xls')
                                new_name = f"{district_name}_{block_name}_{date_download}.xls"
                                source_file = downloads_folder/download_file
                                destination_file = db_input_folder/fps_summary_folder/new_name
                                shutil.move(str(source_file), str(destination_file))
                                print(f"FPS summary {new_name} downloaded successfully.")
                                break # Break out of download_retries
                                
                            except (FileNotFoundError, PermissionError, IsADirectoryError, OSError) as e:
                                print(f"FPS summary for {new_name} could not be downloaded/moved: {e}")
                                break # Break out of download_retries
                                
                            # finally: #Whether file could be moved or not
                            #     break #Break out of download_retries

                        except (StaleElementReferenceException, NoSuchElementException) as e:
                            # print(f"FPS details download button not found for block {block_name} district {district_name}: {e}")
                            print(f"FPS details download button not found for block index {bl_index} district index {dist_index}: {e}")
                            break #Break out of download_retries

                        except (WebDriverException, TimeoutException) as e:
                            # print(f"Error connecting to internt to download FPS details for block {block_name} district {district_name}, retrying: {e}")
                            print(f"Error connecting to internt to download FPS details for block index {bl_index} district index {dist_index}, retrying: {e}")
                            download_retries+=1
                            time.sleep(10)

                    #For each FPS dealer, go to transactions page
                    # for fps_element in block_fps_elements_clickable:
                    for fps_index in range(105, len_block_fps_elements_clickable):
                        block_fps_elements_clickable = wait.until(EC.visibility_of_all_elements_located((By.XPATH, "//a[contains(@onclick, 'detailsRFL')]")))
                        fps_execute_retries = 0
                        fps_success = False
                        while fps_execute_retries<3:
                            try:
                                fps_id = block_fps_elements_clickable[fps_index].text.strip()
                                print(f"FPS ID: {fps_id}")
    
                                #Go to transactions page of each FPS ID
                                # driver.execute_script(fps_element.get_attribute('onclick'))
                                driver.execute_script(block_fps_elements_clickable[fps_index].get_attribute('onclick'))
                                fps_success = True
                                time.sleep(2)
                                break #No more retries if success
    
                            except (StaleElementReferenceException, NoSuchElementException) as e:
                                print(f"Cannot go to transactions page for FPS index {fps_index} in block index {bl_index} district {dist_index}: {e}")
                                break #Break out of while loop
    
                            except (WebDriverException, TimeoutException) as e:
                                print(f"Issue connecting to internet/element click intercepted for loading transactions page for FPS index {fps_index} block index {bl_index} for district {dist_index}, retrying: {e}")
                                fps_execute_retries+=1
                                time.sleep(10)

                            except IndexError as e:
                                print(f"FPS Index {fps_id} out of index in block index {bl_index} district {dist_index} with number of FPS {len_block_fps_elements_clickable}: e")
                                break #Break out of while loop

                        if not fps_success:
                            if fps_execute_retries==3:
                                print(f"Issue connecting to internet/element click intercepted for loading transactions page for FPS index {fps_index} block index {bl_index} for district index {dist_index}: {e}. STOPPED")
                                # driver.close() #Close the driver
                                
                            continue #If could not execute transaction page, go to next FPS
                            
                        #Wait for transactions table rows
                        try:
                            transactions_page = wait.until(EC.visibility_of_element_located((By.XPATH, '//*[@id="Report"]')))
                            # print('Found transactions page')
                            trans_list = []
                            
                            #Select the display in one page button
                            display_retries = 0
                            display_success = False
                            while display_retries<3:
                                try:
                                    display_button = driver.find_element(By.XPATH, '//*[@id="Report_length"]/label/select')
                                    # print(display_button)
                                    display_select = Select(display_button)
                                    display_options = display_select.options
                                    display_select.select_by_value('-1')
                                    time.sleep(1)

                                    try:
                                        footnote = driver.find_element(By.XPATH, '//*[@id="Report_info"]').text.strip()
                                        footnote_parts = footnote.split(' ')
                                        # print(footnote_parts)
                                        if (footnote_parts[3]==footnote_parts[5]):
                                            table_ready = True
                                            # print('Table is ready to be scraped')
                                            display_success = True
                                            break #Break out of display_retries
                                            # break # Break out of 'table retries'
                                        else:
                                            print(f"All pages not displayed for FPS index {fps_index} in block index {bl_index} district index {dist_index}, retrying")
                                            time.sleep(2)
                                            # table_retries+=1
                                            display_retries+=1
                                    
                                    except (StaleElementReferenceException, NoSuchElementException) as e:
                                        print(f"No. of entries shown not found for FPS index {fps_index} in block index {bl_index} district index {dist_index}: {e}")
                                        display_retries+=1
                                        time.sleep(1)
                                    
                                    except (WebDriverException, TimeoutException) as e:
                                        print(f"Error connecting to internet to find no. of entries for FPS index {fps_index} in block index {bl_index} district index {dist_index}: {e}")
                                        display_retries+=1
                                        time.sleep(10)
                                    
                                except (StaleElementReferenceException, NoSuchElementException) as e:
                                    print(f"Display all button could not be found or clicked for FPS index {fps_index} in block index {bl_index} district index {dist_index}, retrying: {e}")
                                    # driver.close()
                                    display_retries+=1
                                    time.sleep(1)
                                    # continue #Continue to next FPS
                                
                                except (WebDriverException, TimeoutException) as e:
                                    print(f"Issue connecting to internet for display all pages for FPS index {fps_index} in block index {bl_index} district index {dist_index}, retrying: {e}")
                                    display_retries+=1
                                    time.sleep(10)
                                    # driver.close()
                            
                            if not display_success:
                                if display_retries==3:
                                    print(f"Display all button could not be found or clicked for FPS index {fps_index} in block index {bl_index} district index {dist_index}: {e}")
                                    print(f"Continuing to next FPS")
                                    time.sleep(1)
                                    continue #Continue to next FPS

                            # #Check if number of pages shown is total number of pages
                            # table_retries = 0
                            # table_ready = False
                            # while table_retries<100:
                            #     try:
                            #         footnote = driver.find_element(By.XPATH, '//*[@id="Report_info"]').text.strip()
                            #         footnote_parts = footnote.split(' ')
                            #         print(footnote_parts)
                            #         if (footnote_parts[3]==footnote_parts[5]):
                            #             table_ready = True
                            #             print('Table is ready to be scraped')
                            #             break # Break out of 'table retries'
                            #         else:
                            #             print(f"All pages not displayed for FPS index {fps_index} in block index {bl_index} district index {dist_index}, retrying")
                            #             time.sleep(5)
                            #             table_retries+=1
                                        
                            #     except (StaleElementReferenceException, NoSuchElementException) as e:
                            #         print(f"No. of entries shown not found for FPS index {fps_index} in block index {bl_index} district index {dist_index}: {e}")
                            #         break # Break out of while loop
                                    
                            #     except (WebDriverException, TimeoutException) as e:
                            #         print(f"Error connecting to internet to find no. of entries for FPS index {fps_index} in block index {bl_index} district index {dist_index}: {e}")
                            #         break # Break out of while loop
                                
                            # if not table_ready:
                            #     if table_retries==100:
                            #         print(f"Table could not be scraped because full page not visible for FPS index {fps_index} in block index {bl_index} district index {dist_index}")
                            #         continue #Move to next FPS
                            #     else:
                            #         continue #If any exception reached
                                    
                            #Find the transactions details again
                            table_retries = 0
                            table_rows_ready = False
                            while table_retries<3:
                                try:
                                    table_rows = wait.until(EC.visibility_of_all_elements_located((By.XPATH, '//*[@id="Report"]/tbody/tr')))
                                    row_length = len([cell.text for cell in table_rows[0].find_elements(By.TAG_NAME, 'td')])
                                    #Set the header
                                    table_header = driver.find_elements(By.XPATH, '//*[@id="Report"]/thead/tr')
                                    header_data_1 = [cell.text for cell in table_header[1].find_elements(By.TAG_NAME, 'th')]
                                    header_data_2 = [cell.text for cell in table_header[2].find_elements(By.TAG_NAME, 'th')]
                                    table_rows_ready=True
                                    break #Break out of while loop
                                
                                except (StaleElementReferenceException, NoSuchElementException) as e:
                                    print(f"No rows 'tr' or header 'th' found for FPS index {fps_index} in block index {bl_index} district index {dist_index}, retrying: {e}")
                                    table_retries+=1
                                    time.sleep(2)

                                except (WebDriverException, TimeoutException) as e:
                                    print(f"Issue connecting to internet for finding table rows for FPS index {fps_index} in block index {bl_index} district index {dist_index}: {e}, retrying")
                                    table_retries+=1
                                    time.sleep(10)

                            if table_rows_ready==False:
                                print(f"Moving to next FPS because could not find rows 'tr' or header 'th' for FPS index {fps_index} in block index {bl_index} district index {dist_index}")
                                continue #Move to next FPS

                            try:
                                try:
                                    quant_index = header_data_1.index('Qty in Kgs')
                                    header_data_1.pop(quant_index)
                                    header_data_1[quant_index:quant_index] = header_data_2
                                    
                                except ValueError:
                                    print(f"No header cell named 'Qty in Kgs' in FPS ID {fps_id} in block {block_name} district {district_name}: {e}")
                                    header_data_1 = [None] * row_length #If there is no header cell called 'Qty in Kgs', header is list of None with length of the first row
                                    
                                header_data_1[0:0] = ['District', 'Block', 'FPS ID', 'Query_Date'] #Add district, block and FPS names to header
                                
                                for row in table_rows:
                                    row_data = [cell.text for cell in row.find_elements(By.TAG_NAME, 'td')]
                                    fps_transactions = [district_name, block_name, fps_id, date] + row_data
                                    trans_list.append(fps_transactions)
                                trans_df = pd.DataFrame(trans_list, columns=header_data_1)
                                trans_file_name = Path(f"{district_name}_{block_name}_{fps_id}.csv")
                                trans_file = db_input_folder/transactions_folder/trans_file_name
                                trans_df.to_csv(trans_file, index=False)
                                print(f"FPS index {fps_index} done.")
                                time.sleep(1)
                            
                            except (WebDriverException, TimeoutException) as e:
                                print(f"Issue connecting to internet for preparing csv file for FPS index {fps_index} in block index {bl_index} district index {dist_index}: {e}")
                                time.sleep(10)
                                # driver.close()
                        
                        except (StaleElementReferenceException, NoSuchElementException) as e:
                            print(f"Transactions page not found for FPS index {fps_index} in block index {bl_index} district index {dist_index}: {e}")
                            continue #Go to next FPS ID

                        except (WebDriverException, TimeoutException) as e:
                            print(f"Issue connecting to internet for loading transactions page for FPS index {fps_index} in block index {bl_index} district index {dist_index}, retrying: {e}")
                            time.sleep(10)
                            # driver.close()
                                                           
                except (StaleElementReferenceException, NoSuchElementException) as e:
                    print(f"No clickable FPS dealer elements in transaction page for block index {bl_index} district index {dist_index}: {e}")
                    continue #Go to FPS dealer page for next block

                except (WebDriverException, TimeoutException) as e:
                    print(f"Issue connecting to internet for loading FPS page for block index {bl_index} district index {dist_index}, retrying: {e}")
                    time.sleep(10)
                    # driver.close() #Close the driver

        except (StaleElementReferenceException, NoSuchElementException) as e:
            print(f"No clickable block elements in page for district index {dist_index}: {e}")
            continue #Continue to next district

        except (WebDriverException, TimeoutException) as e:
            print(f"Issue connecting to internet for loading block page for district index {dist_index}: {e}")
            time.sleep(10)
            # driver.close() #Close the driver

except (StaleElementReferenceException, NoSuchElementException) as e:
    print(f"No clickable district-elements in page for date {date}: {e}")

except (WebDriverException, TimeoutException) as e:
    print(f"Issue connecting to internet for loading district page for date {date},{e}")
    time.sleep(10)
    
# finally:
#     driver.close()
#     print('Closing driver')